In [3]:
import json
import pandas as pd
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('latest_dataset.csv')
df = df.drop('Success',axis=1)

success_data = pd.read_csv("../success_data.csv").drop('Unnamed: 0', axis=1) #, index_col=0)
print(success_data.shape)
upper = success_data['Domestic ROI (%)'].mean() + (3 * success_data['Domestic ROI (%)'].std())
lower = success_data['Domestic ROI (%)'].mean() - (3 * success_data['Domestic ROI (%)'].std())
success_data = success_data[(success_data['Domestic ROI (%)']>=lower) & (success_data['Domestic ROI (%)']<= upper)]
success_data['Success'] = (success_data['Domestic ROI (%)']>70)*1


filterCols = ['passive_ratio', 'pct_coref_sents', 'tot_unique_per_sent','tot_stop_per_sent', 'std_of_overall_polarity', 
              'wav_polarity', 'avg_FK', 'sign_check_char_mention_polairty', 'std_of_char_mention_polarity', 
              'Processed Title', 'avg_sim_score', 'Success']

df = pd.merge(df, success_data, on='Processed Title')

(653, 7)


In [3]:
df = pd.read_csv('latest_dataset.csv')
df = df[filterCols]
df = df.replace('?',np.NaN)
df = df.dropna(how='any')

X = df.drop('Success',axis=1)
y = df['Success']

df.head()

,passive_ratio,pct_coref_sents,tot_unique_per_sent,tot_stop_per_sent,std_of_overall_polarity,wav_polarity,avg_FK,sign_check_char_mention_polairty,std_of_char_mention_polarity,Processed Title,avg_sim_score,Success
0,0.031377,0.054656,2.214575,0.548583,0.778124,0.812752,2.4,0,0.780684,Avatar,0.297514,1
1,0.030601,0.065574,2.373770,0.585792,0.775803,0.683503,3.0,0,0.787237,"Avengers,-The",0.284754,1
2,0.029767,0.185841,2.172969,0.464200,0.784953,0.903355,2.8,0,0.796159,Men-in-Black-3,0.264265,1
3,0.027927,0.060150,2.360902,0.625134,0.871314,-0.294680,2.4,0,0.906034,Guardians-of-the-Galaxy-Vol-2,0.256726,1
4,0.035524,0.046181,2.792185,0.857904,0.701191,0.687086,2.4,0,0.705212,2012,0.174552,1


In [4]:
import omdb
def getOmdbData(movie):
    api_key = 'eb6547ff'
    omdb.set_default('apikey', api_key)
    return omdb.get(title=movie, fullplot=True, tomatoes=True)

files = os.listdir('../scripts/')

#Create a a dict where each item contains information on a movie in dictionary format
movies = {}
for each in files:
    movie = {}
    if '-The_script.txt' in each:
        movies['The '+each.replace('-The_script.txt','').replace('-', ' ')[:-1]] = {'filename':each}
    else:
        movies[each.replace('_script.txt','').replace('-', ' ')] = {'filename':each}



In [5]:
x = getOmdbData('127 Hours')


In [6]:
temp = []
for movie in movies.keys():
    #for the given movie name we fetch the imdb data
    x = getOmdbData(movie)
    temp.append(x)
    
imdb_df = pd.DataFrame(temp)
imdb_df.to_csv('imdb_data.csv',index=False)

In [6]:
df = pd.read_csv('latest_dataset.csv')
imdb_df = pd.read_csv('imdb_data.csv')
feature_df = pd.read_csv('../feat_extraction/movies_with_feats_use_me.csv')
mapping = feature_df[['Unnamed: 0','Processed Title']]
mapping.columns = ['title','Processed Title']
new_df = pd.merge(pd.merge(imdb_df[['title','imdb_rating']], mapping, on='title'), df, on='Processed Title')

In [10]:
new_df['imdb_rating'] <

,imdb_rating,passive_ratio,pct_coref_sents,tot_unique_per_sent,tot_stop_per_sent,std_of_overall_polarity,wav_polarity,avg_FK,sign_check_char_mention_polairty,std_of_char_mention_polarity,avg_sim_score,Success
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,6.963636,0.036045,0.071081,2.243184,0.562857,0.632605,0.297669,2.382592,0.148936,0.617015,0.269754,0.827853
std,0.913758,0.012814,0.044121,0.481752,0.159709,0.361267,0.642070,0.834762,0.430278,0.342068,0.074095,0.377874
min,2.300000,0.009793,0.000000,1.136612,0.202166,0.000185,-0.998798,0.000000,-1.000000,0.000000,0.128296,0.000000
25%,6.400000,0.027829,0.036341,1.931749,0.441702,0.209785,-0.220646,1.800000,0.000000,0.396941,0.217145,1.000000
50%,7.000000,0.033913,0.059791,2.182216,0.537008,0.782361,0.469319,2.400000,0.000000,0.775850,0.256013,1.000000
75%,7.600000,0.042912,0.093750,2.536437,0.656066,0.915044,0.901755,2.800000,0.000000,0.864342,0.305342,1.000000
max,9.300000,0.132075,0.256545,4.457926,1.029354,0.977878,0.999809,5.600000,1.000000,0.979477,0.617082,1.000000


In [28]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_id = X['Processed Title']
X = ss.fit_transform(X.drop('Processed Title',axis=1).values)

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

clfs = {'lr': LogisticRegression(random_state=0),
        'mlp': MLPClassifier(random_state=0),
        'dt': DecisionTreeClassifier(random_state=0),
        'rf': RandomForestClassifier(random_state=0),
        'svc': SVC(random_state=0)}

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe_clfs = {}

for name, clf in clfs.items():
    # Implement me
    pipe_clfs[name] = Pipeline([('StandardScaler',StandardScaler()),('clf',clf)])

In [31]:
param_grids = {}

In [32]:
C_range = [10 ** i for i in range(-4, 5)]

param_grid = [{'clf__multi_class': ['ovr'], 
               'clf__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
               'clf__C': C_range},
              {'clf__multi_class': ['multinomial'],
               'clf__solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
               'clf__C': C_range}]

# Implement me
param_grids['lr'] = param_grid

In [33]:
param_grid = [{'clf__hidden_layer_sizes': [10, 100, 200],
               'clf__activation': ['identity', 'logistic', 'tanh', 'relu']}]

# Implement me
param_grids['mlp'] = param_grid

In [34]:
param_grid = [{'clf__min_samples_split': [2, 10, 30],
               'clf__min_samples_leaf': [1, 10, 30]}]

# Implement me
param_grids['dt'] = param_grid

In [35]:
param_grid = [{'clf__n_estimators': [2, 10, 30],
               'clf__min_samples_split': [2, 10, 30],
               'clf__min_samples_leaf': [1, 10, 30]}]

# Implement me
param_grids['rf'] = param_grid

In [36]:
param_grid = [{'clf__C': [0.01, 0.1, 1, 10, 100],
               'clf__gamma': [0.01, 0.1, 1, 10, 100],
               'clf__kernel': ['linear', 'poly', 'rbf', 'sigmoid']}]

# Implement me
param_grids['svc'] = param_grid

In [37]:
X

array([[-0.36238155, -0.374653  , -0.05975787, ..., -0.35228279,
         0.48016592,  0.37777427],
       [-0.42232269, -0.12792826,  0.26584147, ..., -0.35228279,
         0.49930884,  0.20436729],
       [-0.48682259,  2.5898864 , -0.1448543 , ..., -0.35228279,
         0.52536775, -0.07406607],
       ...,
       [-0.60870943, -1.03979261, -0.45613979, ..., -0.35228279,
         0.3213868 , -0.74758159],
       [ 0.31213687, -0.1829286 , -0.80401902, ..., -0.35228279,
         0.52836867,  1.42863646],
       [ 0.83393843,  0.92807414, -0.97995141, ...,  1.96838006,
        -1.79779358,  0.87720494]])

In [38]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [70]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# The list of [best_score_, best_params_, best_estimator_]
best_score_param_estimators = []

# For each classifier
for name in pipe_clfs.keys():
    # GridSearchCV
    # Implement me
    gs = GridSearchCV(estimator=pipe_clfs[name], param_grid=param_grids[name], scoring='precision', n_jobs=-1, cv=StratifiedKFold(
    n_splits=10, shuffle=True, random_state=10))
    
    # Fit the pipeline
    gs.fit(X_train, y_train)
    
    # Update best_score_param_estimators
    best_score_param_estimators.append([gs.best_score_, gs.best_params_, gs.best_estimator_])

/Users/akartikay/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/akartikay/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/akartikay/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/akartikay/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationW

In [71]:
best_score_param_estimators = sorted(best_score_param_estimators, key=lambda x : x[0], reverse=True)

# For each [best_score_, best_params_, best_estimator_]
for best_score_param_estimator in best_score_param_estimators:
    # Print out [best_score_, best_params_, best_estimator_], where best_estimator_ is a pipeline
    # Since we only print out the type of classifier of the pipeline
    print([best_score_param_estimator[0], best_score_param_estimator[1], type(best_score_param_estimator[2].named_steps['clf'])], end='\n\n')

[1.0, {'clf__C': 1, 'clf__gamma': 100, 'clf__kernel': 'rbf'}, <class 'sklearn.svm.classes.SVC'>]

[0.6762785296350886, {'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 2}, <class 'sklearn.ensemble.forest.RandomForestClassifier'>]

[0.6058917704614446, {'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2}, <class 'sklearn.tree.tree.DecisionTreeClassifier'>]

[0.6006057245405045, {'clf__activation': 'relu', 'clf__hidden_layer_sizes': 200}, <class 'sklearn.neural_network.multilayer_perceptron.MLPClassifier'>]

[0.546901213756852, {'clf__C': 0.0001, 'clf__multi_class': 'multinomial', 'clf__solver': 'newton-cg'}, <class 'sklearn.linear_model.logistic.LogisticRegression'>]



In [72]:
pd.crosstab(y_test, best_score_param_estimators[0][-1].predict(X_test), rownames=['True'], colnames=['Predicted'])
best_score_param_estimators[0][2].fit(X_train, y_train)
print(best_score_param_estimators[0][2].score(X_test, y_test))
print(X_train.shape, y_train.shape)
best_score_param_estimators[0][2].predict(X_test)

0.7142857142857143
(439, 10) (439,)


array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0])

In [73]:
pd.crosstab(y_test, best_score_param_estimators[0][-1].predict(X_test), rownames=['True'], colnames=['Predicted'])

Predicted,0,1
True,,
0,95,0
1,54,40


In [74]:
from imblearn.over_sampling import RandomOverSampler

In [75]:
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_sample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_ros, y_ros, test_size=0.3, random_state=0, stratify=y_ros)

In [76]:
X_ros, y_ros = ros.fit_sample(X, y)

In [77]:
X_ros.shape

(628, 10)

In [78]:
y_ros.shape

(628,)

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X_ros, y_ros, test_size=0.3, random_state=0, stratify=y_ros)

In [80]:
best_score_param_estimators[0][-1].predict(X_test)

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [81]:
pd.crosstab(y_test, best_score_param_estimators[0][-1].predict(X_test), rownames=['True'], colnames=['Predicted'])

Predicted,0,1
True,,
0,95,0
1,57,37
